In [1]:
import torch
import numpy as np
import random

#Certifying ResNet_4B requires large GPU memory, I would suggest moving to the CPU
# device = 'cuda:0' if torch.cuda.is_available else 'cpu'
device = 'cpu'

In [2]:
from auto_LiRPA import BoundedModule, PerturbationLpNorm, BoundedTensor
from auto_LiRPA.utils import get_spec_matrix
from cert_util import min_correct_with_eps, load_data, DeltaWrapper

# from model_defs import resnet2b, resnet4b
from resnext import ResNeXt_cifar 

In [3]:
cifar10_mean = (0.4914, 0.4822, 0.4465)  # np.mean(train_set.train_data, axis=(0,1,2))/255
cifar10_std = (0.2471, 0.2435, 0.2616)  # np.std(train_set.train_data, axis=(0,1,2))/255

mu = torch.tensor(cifar10_mean).view(3,1,1).to(device)
std = torch.tensor(cifar10_std).view(3,1,1).to(device)

def normalize(X, device):
    result = (X.to(device) - mu.to(device))/std.to(device)
    return result

def bounded_results(eps,bounded_model):
    # normalization for the input noise
    ptb = PerturbationLpNorm(x_L=-eps/std.view(1,3,1,1).expand(1,3,32,32), x_U=eps/std.view(1,3,1,1).expand(1,3,32,32))
    bounded_delta = BoundedTensor(delta, ptb)
    
    final_name = bounded_model.final_name
    input_name = '/1' # '/input.1' 

    result = bounded_model.compute_bounds(
        x=(new_image,bounded_delta), method='backward', C=C,
        return_A=True, 
        needed_A_dict={ final_name: [input_name] },
    )
    lower, upper, A_dict = result
    lA = A_dict[final_name][input_name]['lA']
    uA = A_dict[final_name][input_name]['uA']

    lb = lower - ptb.concretize(delta, lA, sign=-1)
    ub = upper - ptb.concretize(delta, uA, sign=1)


    lA = torch.reshape(lA,(eval_num, num_cls-1,-1))
    return lA,lb, lower

### Reproducibility

In [4]:
my_seed = 4
torch.cuda.empty_cache()
torch.manual_seed(my_seed)
random.seed(my_seed)
np.random.seed(my_seed)

### Hyperparameters 

In [5]:
eval_num = 5
num_cls = 10
adv_e = 3

# CIFAR models, one can choose from 'resnet2b', 'resnet4b'
model_name = "ResNeXt_cifar" 
# adv_train_norm = 8

### Loadding the model for certification

In [6]:
net = eval(model_name)()
net.load_state_dict(torch.load('./model_weights/ResNeXt_cifar',map_location=device)['state_dict'])
net.to(device)

ResNeXt(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (layer1): Sequential(
    (0): Block(
      (conv1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=2)
      (conv3): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
      (shortcut): Sequential(
        (0): Conv2d(16, 128, kernel_size=(1, 1), stride=(1, 1))
      )
    )
  )
  (layer2): Sequential(
    (0): Block(
      (conv1): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1))
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=2)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1))
      (shortcut): Sequential(
        (0): Conv2d(128, 256, kernel_size=(1, 1), stride=

### Loading a batch of data

In [7]:
new_image, new_label = load_data(num_imgs=eval_num, random=True, dataset='CIFAR')
new_image = normalize(new_image,device)
C = get_spec_matrix(new_image,new_label.long(),10)

Files already downloaded and verified


In [8]:
print('Validating the model ACC:', torch.sum(torch.argmax(net(new_image),axis=1) == new_label.to(device)).detach().cpu().numpy()/new_label.shape[0])

Validating the model ACC: 0.6


### Model concretization

In [9]:
import warnings
warnings.filterwarnings('ignore')

eps = adv_e/255
delta = torch.zeros_like(new_image[0]).unsqueeze(0)
dummy_input = (new_image, delta)
model = DeltaWrapper(net.to(device))
bounded_model = BoundedModule(model, dummy_input, bound_opts={'conv_mode':'matrix'})
bounded_model.eval()
final_name = bounded_model.final_name

### Results and comparision 

In [10]:
print('Eps:', adv_e)

alpha,beta,result = bounded_results(eps,bounded_model)
samp_ACC = torch.sum(result.detach().cpu().min(axis=1)[0] > 0).numpy()/result.shape[0]
print('Samp-wise Cert-ACC: {}%'.format(samp_ACC*100))

label = new_label
number_class = num_cls
cert_ACC, delta = min_correct_with_eps(alpha, beta, eps, label, number_class=10, verbose=False, dataset='CIFAR')
print('UP-based Cert-ACC: {}%'.format(100*cert_ACC/label.shape[0]))

Eps: 3
Samp-wise Cert-ACC: 60.0%
Set parameter Username
Academic license - for non-commercial use only - expires 2023-05-24
Set parameter TimeLimit to value 600
UP-based Cert-ACC: 60.0%
